# This model finds faces with CNN

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

I use face recognition library to recognize faces. I use fastai.plots to plot the results to see the photos on which the algorithm found face and on which it didn't

In [ ]:
import face_recognition
from fastai.plots import *
import os
import itertools

In [ ]:
PATH = "your_path_here"

I structured this the following way. In the main folder there are photos of a person and only him on each photo. The rest folder contains a set of pictures to sort.

In [ ]:
main_photos = find_all_images(f"{PATH}/main")
root_dir = find_all_images(f"{PATH}/the_rest")

In [ ]:
def find_all_images(root_dir):
    """
        Finds all images in a given folder, including in subdirectories.
    """
    all_images = []
    for path, subdirs, files in os.walk(root_dir):
        for name in files:
            f_name = os.path.join(path, name)
            if f_name[-4:].lower() == ".jpg" or f_name[-4:].lower() == ".png":
                all_images.append(f_name)
    return all_images

def find_persons_encodings(photos):
    """
        Finds face encodings for the main faces from different photos if there are more than one
    """
    persons_face_encodings = []
    for person in photos:
        picture_of_person = face_recognition.load_image_file(person)
        face_location = face_recognition.face_locations(picture_of_person, number_of_times_to_upsample=0, model="cnn")
        t = face_recognition.face_encodings(picture_of_person, face_location)
        if len(t) != 0:
            persons_face_encodings.append(t[0])
    return persons_face_encodings

def find_face_encodings(all_imgs):
    """
        Finds face encodings for all faces from all photos from "the rest" folder
    """
    unknown_face_encodings = []
    for fname in all_imgs:
        unknown_picture = face_recognition.load_image_file(fname)
        unknown_face_locations = face_recognition.face_locations(unknown_picture, number_of_times_to_upsample=0, model="cnn")
        if len(unknown_face_locations) == 0:
            unknown_face_encodings.append(-1)
        else:
            unknown_face_encodings.append(face_recognition.face_encodings(unknown_picture, 
                                                                      known_face_locations=unknown_face_locations))
    return unknown_face_encodings

In [ ]:
def find_matches(root_dir, main_photos, unknown_face_encodings):
    """
        root_dir -> root directory of all the images
        main_photos -> list of file names each one of which contain a picture of a person for encoding
    """
    matched_faces = []
    not_matched_faces = []
    persons_face_encodings = find_persons_encodings(main_photos)
        
    for index, faces_enc in enumerate(unknown_face_encodings):
        if faces_enc == -1:
            not_matched_faces.append(root_dir[index])
            continue
        flag = 0
        for main_face_enc in persons_face_encodings:
            results = face_recognition.compare_faces(faces_enc, main_face_enc)
            if True in results:
                flag = 1
                matched_faces.append(root_dir[index])
                break
        if flag == 0:
            not_matched_faces.append(root_dir[index])
    
    return (matched_faces, not_matched_faces)

Find encodings for all faces

In [ ]:
unknown_face_encodings = find_face_encodings(root_dir)

two lists of matched and not_matched photos

In [ ]:
matched, not_matched = find_matches(root_dir, main_photos, unknown_face_encodings)

In [ ]:
(len(matched), len(not_matched))

In [ ]:
coeff_m = len(matched)//4 + 1
coeff_n = len(not_matched)//4 + 1

In [ ]:
plots_from_files(matched,figsize=(25, coeff_m * 4), rows=coeff_m)

In [ ]:
plots_from_files(not_matched,figsize=(25, coeff_n * 4), rows=coeff_n )